<a href="https://colab.research.google.com/github/cr21/Unsupervised-Machine-Learning-Clustering/blob/main/FeatureSelectionChi2Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
cd drive/MyDrive/Dataset/

[Errno 2] No such file or directory: 'drive/MyDrive/Dataset/'
/content/drive/MyDrive/Dataset


In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection  import mutual_info_classif

# Feature Selection using Chi2 Test

In [6]:
# Load 20 News Group dataset and split train and test data
def load20NG():
  dataset = fetch_20newsgroups(subset='all', remove=('headers','footers','quotes'), random_state= 12)
  data = dataset.data
  labels = dataset.target

  vectorizer = TfidfVectorizer(stop_words='english',min_df = 5)

  # # vectorize train data
  ngVectorData = vectorizer.fit_transform(data)
  featureNames = vectorizer.get_feature_names()
  return data, labels, ngVectorData, featureNames
data, labels, ng20Data, ng20featureNames = load20NG()
X_train, X_test, y_train, y_test = train_test_split(ng20Data, labels, test_size=0.33, random_state=42)

In [48]:
X_train.shape

(12626, 24164)

In [54]:
Transformed_200_X_train = SelectKBest(chi2, k=200).fit_transform(X_train, y_train)

In [55]:
Transformed_200_X_test = SelectKBest(chi2, k =200).fit_transform(X_test,y_test)

In [58]:
ngClassfier = LogisticRegression(penalty='l2', tol =0.0001, C = 1.0,n_jobs=-1)
ngClassfier.fit(Transformed_200_X_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [59]:
# training error
print(f" training Error : {ngClassfier.score(Transformed_200_X_train, y_train)}")

 training Error : 0.5355615396800253


In [60]:
#testing error
print(f" testing Error : {ngClassfier.score(Transformed_200_X_test, y_test)}")

 testing Error : 0.09212218649517685


# Mutual Information for Feature Selection

In [90]:
feature_scores = mutual_info_classif(X_train, y_train, random_state=0)

In [ ]:
# Top 200 features
sorted_ngFeatureList = np.argsort(feature_scores)
reverse_sorted_ngFeatureList = sorted_ngFeatureList[::-1]
top200 = reverse_sorted_ngFeatureList[:200]
for id, featureId in enumerate(top200):
  featureNameList = np.array(ng20featureNames)[featureId]
  print(f" class : {id} , feature name : {featureNameList}")


In [108]:
top200XTrain = X_train[:,top200 ]
top200XTest = X_test[:, top200]

In [109]:
ngClassfier = LogisticRegression(penalty='l2', tol =0.0001, C = 1.0,n_jobs=-1)
ngClassfier.fit(top200XTrain, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [110]:
# training error
print(f" training Error : {ngClassfier.score(top200XTrain, y_train)}")

 training Error : 0.3446063678124505


In [111]:
#testing error
print(f" testing Error : {ngClassfier.score(top200XTest, y_test)}")

 testing Error : 0.30884244372990355
